# Exam : Segmenting and Clustering Neighborhoods in Toronto

## Scrapping and preparation of dataframe

In [1]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
!pip install lxml

In [3]:
import requests

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())
wiki_tab = soup.find('table',{'class':'wikitable sortable'})
#wiki_tab = wiki_tab.find_all('tr')
#print(wiki_tab)

Now that the wiki_table is extracted lets extract informations from colomns and line in order to create the dataframe


In [4]:
#Initialize datafram
postcode=[]
borough=[]
neighbourhood=[]

for col in range(len(wiki_tab.findAll('tr'))):
    for line in range(len(wiki_tab.findAll('tr')[col].findAll('td'))):
        if line == 0 :
            postcode.append(str(wiki_tab.findAll('tr')[col].findAll('td')[line].getText()))
        elif line == 1 :
            borough.append(str(wiki_tab.findAll('tr')[col].findAll('td')[line].getText()))
        elif line == 2 :
            neighbourhood.append(str(wiki_tab.findAll('tr')[col].findAll('td')[line].getText()).strip('\n'))



In [49]:
wiki_data={'PostalCode':postcode,'Borough':borough,'Neighbourhood':neighbourhood}
wiki_df=pd.DataFrame(data=wiki_data)
wiki_df


,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


## Cleaning and requirements

1. Ignore cells with a borough that is Not assigned


In [61]:
wiki_data={'PostalCode':postcode,'Borough':borough,'Neighbourhood':neighbourhood}
wiki_df=pd.DataFrame(data=wiki_data)
not_assigned=wiki_df['Borough']!='Not assigned'
wiki_df=wiki_df[not_assigned]

In [62]:
print(wiki_df)

    PostalCode           Borough             Neighbourhood
2          M3A        North York                 Parkwoods
3          M4A        North York          Victoria Village
4          M5A  Downtown Toronto              Harbourfront
5          M6A        North York          Lawrence Heights
6          M6A        North York            Lawrence Manor
..         ...               ...                       ...
281        M8Z         Etobicoke  Kingsway Park South West
282        M8Z         Etobicoke                 Mimico NW
283        M8Z         Etobicoke        The Queensway West
284        M8Z         Etobicoke     Royal York South West
285        M8Z         Etobicoke            South of Bloor

[210 rows x 3 columns]


2. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [79]:
df_3=wiki_df
df_3=df_3.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()
df_3

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern,Rouge,Malvern,Rouge,Malvern,Roug..."
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,Highland ..."
2,M1E,Scarborough,"Guildwood,Morningside,West Hill,Guildwood,Morn..."
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


3.If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [92]:
not_assigned_neigh=df_3['Neighbourhood']=='Not assigned'
df_3.loc[not_assigned_neigh,'Borough']=df_3.loc[not_assigned_neigh,'Neighbourhood']
df_3
    

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern,Rouge,Malvern,Rouge,Malvern,Roug..."
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,Highland ..."
2,M1E,Scarborough,"Guildwood,Morningside,West Hill,Guildwood,Morn..."
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [96]:
df_3.shape

(103, 3)

Thank you for reading 